In [4]:
import galois
import numpy as np
MOD = 7514777789

points = []

for line in open('C:\\Users\\Admin\\Downloads\\encoded.txt', 'r').read().strip().split('\n'):
    x, y = line.split(' ')
    points.append((int(x), int(y)))

GF = galois.GF(MOD)

matrix = []
solution = []
for point in points:
    x, y = point
    solution.append(GF(y % MOD))

    row = []
    for i in range(len(points)):
        row.append(GF((x ** i) % MOD))
    
    matrix.append(GF(row))

open('output.bmp', 'wb').write(bytearray(np.linalg.solve(GF(matrix), GF(solution)).tolist()[:-1]))


KeyboardInterrupt: 

In [23]:
p = 7514777789-1

X = []
Y = []
for line in open('C:\\Users\\Admin\\Downloads\\encoded.txt', 'r').read().strip().split('\n'):
    x, y = line.split(' ')
    X.append(int(x))
    Y.append(int(y))

try:
    K = GF(p)
    R = PolynomialRing(K, 'x')
except NameError:
    raise Exception("SageMath kernel is required to run this script!")

Exception: SageMath kernel is required to run this script!

In [20]:
import multiprocessing as mp
import galois
import numpy as np

MOD = 7514777789
p = MOD  # Sử dụng giá trị hợp lệ cho phần tử trường hữu hạn

X = []
Y = []
with open('C:\\Users\\Admin\\Downloads\\encoded.txt', 'r') as file:
    for line in file.read().strip().split('\n'):
        x, y = line.split(' ')
        X.append(int(x))
        Y.append(int(y) % MOD)  # Điều chỉnh y để đảm bảo nó nằm trong phạm vi MOD

# Khởi tạo trường hữu hạn GF(MOD)
K = galois.GF(MOD)

def compZ(X):
    Z = K(1)
    for xk in X:
        Z *= galois.Poly([-xk, 1], field=K)  # Khởi tạo đa thức cho (x - xk)
    return Z

def comp(X, Y, Xother):
    Z = compZ(Xother)
    Y = [K(y) / Z(K(x)) for x, y in zip(X, Y)]  # Sửa lại phép chia
    return Y, Z

def solve(X, Y):
    n = len(Y)
    print("Solving for", n, "points...")

    if n <= 10:
        return galois.lagrange_poly(X, Y, field=K)

    nhalf = n // 2

    X1, Y1 = X[:nhalf], Y[:nhalf]
    X2, Y2 = X[nhalf:], Y[nhalf:]

    if nhalf > 10000:
        with mp.Pool(2) as pool:
            result1 = pool.apply_async(comp, (X1, Y1, X2))
            result2 = pool.apply_async(comp, (X2, Y2, X1))

            Y1, Z2 = result1.get()
            Y2, Z1 = result2.get()
    else:
        Y1, Z2 = comp(X1, Y1, X2)
        Y2, Z1 = comp(X2, Y2, X1)

    f1 = solve(X1, Y1)
    f2 = solve(X2, Y2)

    return f1 * Z2 + f2 * Z1

if __name__ == "__main__":
    def test():
        Xt = X[:1000]
        Yt = Y[:1000]
        f = solve(Xt, Yt)
        for x, y in zip(Xt, Yt):
            assert f(x) == y

    test()

    f = solve(X, Y)

    with open("output.bmp", "wb") as output_file:
        output_file.write(bytearray(f.coefficients(sparse=False)[:-1]))


Solving for 1000 points...
Solving for 500 points...
Solving for 250 points...
Solving for 125 points...
Solving for 62 points...
Solving for 31 points...
Solving for 15 points...
Solving for 7 points...


TypeError: lagrange_poly() got an unexpected keyword argument 'field'